#Neural Network (TensorFlow/Keras)

In [ ]:
!sudo pip3 install keras

1: **Setup and Data Loading**

- **Imports**:

- - tensorflow and keras for building/training the neural network.

- - StandardScaler for feature standardization.

- - RandomOverSampler to handle class imbalance.

- **Data Loading**:

- Loads the magic04.data file with particle collision features.

- Converts the class column to binary (1=Gamma, 0=Hadron).

In [ ]:
# Setup and Data Loading
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Load data
cols = ["fLength", "fWidth", "fSize", "fConc", "fConcl", "fAsym",
        "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv("magic04.data", names=cols)
df["class"] = (df["class"] == "g").astype(int)  # Binary: 1=Gamma, 0=Hadron

2: **Data Preparation**

- **prepare_data() Function**:

- Standardizes features using StandardScaler() (critical for neural networks).

- If oversample=True, applies RandomOverSampler to balance classes.

In [ ]:
# Data Preparation
def prepare_data(df, oversample=False):
    X = df[cols[:-1]].values
    y = df[cols[-1]].values
    X = StandardScaler().fit_transform(X)  # Standardize features
    if oversample:
        from imblearn.over_sampling import RandomOverSampler
        X, y = RandomOverSampler().fit_resample(X, y)
    return X, y

3: **Data Splitting**

- **Splits Data**:

- - 60% training, 20% validation, 20% test (shuffled).

- - Training set is oversampled; validation/test sets use original distribution.

In [ ]:
# Data Splitting - FIXED VERSION
# Split into train (60%), validation (20%), and test (20%)
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
X_train, y_train = prepare_data(train, oversample=True)  # Balance classes
X_valid, y_valid = prepare_data(valid)                   # Validation set
X_test, y_test = prepare_data(test)                      # Test set

4: **Neural Network Model**

- **Architecture**:

- Input Layer: 10 features → 32 neurons (ReLU activation).

- Hidden Layer: 32 neurons (ReLU).

- Output Layer: 1 neuron (sigmoid for binary probability).

- **Compilation**:

- - Optimizer: Adam (adaptive learning rate).

- - Loss: binary_crossentropy (for binary classification).

- - Metric: Accuracy.

In [ ]:
# Neural Network Model
model = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(10,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

5: **Training with Validation**

- **Training**:

- - Runs for 50 epochs (passes through the dataset).

- - Batch Size: 32 (samples processed before weight updates).

- - Validation Data: Monitored to detect overfitting.

In [ ]:
# Training with Validation
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_valid, y_valid),  # Now properly defined
                    verbose=1)


6: **Evaluation**

- **Test Performance**:

- - Computes loss and accuracy on the unseen test set.

- - Example output: Test accuracy: 0.8724.

In [ ]:
# Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest accuracy: {test_acc:.4f}")